In [38]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from copy import deepcopy

from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from collections import defaultdict
from skopt.plots import plot_convergence

import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
#Validation function
def rmsle_cv(model, X_train, y_train, n_folds):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    clf = cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf)
    rmse= np.sqrt(-clf)
    return(rmse)

In [40]:
test_df = pd.read_csv('../data_out/temp_test.csv', parse_dates=['DateTime'], index_col='DateTime')
train_df = pd.read_csv('../data_out/temp_train.csv', parse_dates=['DateTime'], index_col='DateTime')

In [41]:
test_df.head()

,Fc,Ta,Ws,Fg,VPD,Fn,q,Ts,Sws,EVI,Set_rank
DateTime,,,,,,,,,,,
2013-03-11 16:30:00,NaN,23.071327,7.011481,54.543833,1.019578,297.580602,0.011173,30.070280,0.034217,0.212999,test
2013-03-11 17:00:00,NaN,21.780315,7.195028,51.840274,0.779860,222.707777,0.011336,29.996108,0.034211,0.213017,test
2013-03-11 17:30:00,NaN,20.938351,7.195028,28.480794,0.682643,129.255801,0.011120,29.551076,0.034208,0.213035,test
2013-03-11 18:00:00,NaN,20.096387,7.195028,5.121314,0.591581,35.803825,0.010903,29.106044,0.034204,0.213052,test
2013-03-11 18:30:00,NaN,19.366684,6.424132,-9.284288,0.543268,-5.318600,0.010552,28.586839,0.034202,0.213070,test


In [42]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1441 entries, 2013-03-11 16:30:00 to 2013-04-10 16:30:00
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Fc        1030 non-null   float64
 1   Ta        1441 non-null   float64
 2   Ws        1441 non-null   float64
 3   Fg        1441 non-null   float64
 4   VPD       1441 non-null   float64
 5   Fn        1441 non-null   float64
 6   q         1441 non-null   float64
 7   Ts        1441 non-null   float64
 8   Sws       1441 non-null   float64
 9   EVI       1441 non-null   float64
 10  Set_rank  1441 non-null   object 
dtypes: float64(10), object(1)
memory usage: 135.1+ KB


In [43]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5807 entries, 2013-03-03 00:00:00 to 2013-07-31 23:30:00
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Fc        4218 non-null   float64
 1   Ta        5807 non-null   float64
 2   Ws        5807 non-null   float64
 3   Fg        5807 non-null   float64
 4   VPD       5807 non-null   float64
 5   Fn        5807 non-null   float64
 6   q         5807 non-null   float64
 7   Ts        5807 non-null   float64
 8   Sws       5807 non-null   float64
 9   EVI       5807 non-null   float64
 10  Set_rank  5807 non-null   object 
dtypes: float64(10), object(1)
memory usage: 544.4+ KB


In [44]:
train_df.head()

,Fc,Ta,Ws,Fg,VPD,Fn,q,Ts,Sws,EVI,Set_rank
DateTime,,,,,,,,,,,
2013-03-03 00:00:00,NaN,17.402101,0.734186,-41.948923,0.066992,-51.788283,0.011947,26.880882,0.029436,0.205680,layer2
2013-03-03 00:30:00,NaN,16.728530,1.137989,-40.050291,-0.000369,-51.275960,0.011853,26.510021,0.029436,0.205698,layer2
2013-03-03 01:00:00,NaN,16.054958,1.541792,-38.151658,-0.063359,-50.763637,0.011750,26.139161,0.029436,0.205715,layer2
2013-03-03 01:30:00,NaN,15.381387,1.321536,-39.019591,-0.082411,-51.080736,0.011390,25.768301,0.029436,0.205733,layer2
2013-03-03 02:00:00,NaN,14.707816,1.101280,-39.887525,-0.100487,-51.397834,0.011041,25.397440,0.029436,0.205750,layer2


In [45]:
# This should come from the config

Xvar = ['Ta', 'Ws', 'Fg', 'VPD', 'Fn', 'q', 'Ts', 'Sws', 'EVI']
yvar = 'Fc'

solvers_layer1 = ['LGBM', 'RFE', 'SVM', 'GP', 'ANN', 'LASSO']
solvers_layer2 = ['LGBM']

In [46]:
# Combining data frame for scaling
full_df = pd.concat([train_df, test_df])
ymean, ystd = full_df[yvar].mean(), full_df[yvar].std()
full_df[yvar] = (full_df[yvar] - ymean)/ystd

In [47]:
full_df.shape

(7248, 11)

In [48]:
full_df['Set_rank'].value_counts()

layer2            2904
test              1441
layer1_subset6     488
layer1_subset2     483
layer1_subset1     483
layer1_subset3     483
layer1_subset4     483
layer1_subset5     483
Name: Set_rank, dtype: int64

In [49]:
# Layer 1 training

N_FOLDS = 3
N_CALLS = 51
THRESHOLD = 0.05
SCORES = {}

In [50]:
### Custom cross-validation function: weighted F1 score on specific threshold
def objective_core(cls, Xtrain, ytrain, label, **params):
    """
    Objective function to minimize, using custom cross-validation. Default CV is limited 
    """
    cls_ = cls
    Xtrain = Xtrain.copy()
    ytrain = ytrain.copy()

    # Update parameters with default parameters
    params = {**params, **{'n_jobs':-1}}    

    print("------ Sampling new data point ------")

    kfold = KFold(n_splits=N_FOLDS, shuffle=True)
    cval_results = defaultdict(list)
    all_scores = defaultdict(list)
    for train_index, test_index in kfold.split(Xtrain, ytrain):
        X_train_, X_test_ = Xtrain.iloc[train_index, :], Xtrain.iloc[test_index, :]
        y_train_, y_test_ = ytrain.iloc[train_index], ytrain.iloc[test_index]

        # Train regressor on current fold
        cls_.fit(X_train_, y_train_)
        y_test_predicted = cls_.predict(X_test_)


        cval_results['mse'].append(mean_squared_error(y_test_ * ystd + ymean, 
                                                  y_test_predicted * ystd + ymean))


        ametric = diagnostic_stats(y_test_ * ystd + ymean, y_test_predicted * ystd + ymean)
        for k in ['rmse', 'rsqr', 'mbe']:
            all_scores[k].append(ametric[k])
        
    score = np.mean(np.sqrt(cval_results['mse']))
    
    record_score = {}
    for k in ['rmse', 'rsqr', 'mbe']:
        record_score[k] = np.mean(all_scores[k])
    SCORES[label] = record_score
    
    print("Params:",params)
    print("Score:", score)
    
    return score

In [51]:
# List of models

model_library = { 
              'LGBM': 
                 {
                     'params_space': [Integer(2, 10, name='num_leaves'),
                                      Categorical(['regression'], name = "objective"),
                                      Integer(2, 10, name='min_data_in_leaf'),
                                      Real(10**-4, 10**0, "uniform", name='learning_rate'),
                                      Integer(100, 500, name='n_estimators')],
                      'model_instance': lgb.LGBMRegressor(),
                      'data': 'subset1'},
              
              'RFE':
                  {
                      'params_space': [Integer(2, 25, name='max_depth'),
                                       Integer(2, 15, name='min_samples_leaf'),
                                       Integer(2, 15, name='min_samples_split'),
                                       Integer(100, 500, name='n_estimators')],
                      'model_instance': RandomForestRegressor(),
                      'data': 'subset2'
                  },
              
              'SVM':
                  {
                      'params_space': [Integer(2, 6, name='degree'),
                                     Categorical(['scale'], name='gamma'),
                                     Categorical(['rbf', 'poly', 'sigmoid'], name='kernel'),
                                     Real(1, 100, "uniform", name='C')],
                      'model_instance': SVR(),
                      'data': 'subset3'
                  },
              
              'GP':
                  {
                      'params_space': [Categorical([None], name='kernel'),
                                       Categorical(['fmin_l_bfgs_b', 'adam'], name='optimizer'),
                                       Real(1e-4, 1, "uniform", name='alpha')],
                      'model_instance': GaussianProcessRegressor(),
                      'data': 'subset4'
                  },
              
              'ANN':
                  {
                      'params_space': [Integer(2, 20, name='hidden_layer_sizes'),
                                         Categorical(['relu'], name='activation'),
                                         Categorical(['adam','lbfgs'], name='solver'),
                                         Real(1e-4, 1, "uniform", name='alpha'),
                                         Real(1e-3, 0.1, "uniform", name='learning_rate_init'),
                                         Categorical(['constant', 'adaptive'], name='learning_rate')],
                      'model_instance': MLPRegressor(),
                      'data': 'subset5'
                  },
    
              'LASSO':
                 {'params_space': None, 'model_instance': Lasso(), 'data': 'subset6'}
             }

In [52]:
for key in solvers_layer1:
    label = f'Layer1_{key}'
    val = deepcopy(model_library[key])
    subset = val['data']
    reg = val['model_instance']
    params_space = val['params_space']

    print(f'Layer1 {key}: Data: {subset}')
    
    # Train/test split. Ignoring NaN yvar in training set
    train_layer1_df = full_df[full_df['Set_rank']==f'layer1_{subset}'].copy()
    train_layer1_df = train_layer1_df[~train_layer1_df[yvar].isna()]
    X_train_layer1 = train_layer1_df[Xvar]
    y_train_layer1 = train_layer1_df[yvar]

    if key=='LASSO':
        ith_model = make_pipeline(RobustScaler(), Lasso(alpha =1e-4, random_state=1))
        score = rmsle_cv(ith_model, X_train_layer1, y_train_layer1, N_FOLDS)
        print("\nLasso score: {:.4f} (+/-{:.4f})\n".format(score.mean(), score.std()))

    else:
        # Bayesian opt. part
        @use_named_args(params_space)
        def ith_objective(**params):
            cls = reg.set_params(**params)
            return objective_core(cls, X_train_layer1, y_train_layer1, label, **params)

        res = gp_minimize(ith_objective, params_space, n_calls=N_CALLS, random_state=0)
        "Best score=%.4f" % res.fun

        # Generating final optimized model instance
        print("Optimal parameters")
        params = {}
        for param, value in zip(params_space, res.x):
            print(f"Param: {param.name}, value: {value}")
            params[param.name] = value

        ith_model = reg.set_params(**params)
    
    ith_model.fit(X_train_layer1.values, y_train_layer1.values)
    
    # Overriding model instance by fitted model
    model_library[key]['model_instance_layer1'] = ith_model

Layer1 LGBM: Data: subset1
------ Sampling new data point ------
RMSE: 2.531, R^2: 0.470, MBE: -0.213
RMSE: 3.242, R^2: 0.273, MBE: -0.074
RMSE: 3.223, R^2: 0.335, MBE: 0.496
Params: {'num_leaves': 7, 'objective': 'regression', 'min_data_in_leaf': 9, 'learning_rate': 0.8472670136102473, 'n_estimators': 349, 'n_jobs': -1}
Score: 2.998914070765942
------ Sampling new data point ------
RMSE: 3.021, R^2: 0.395, MBE: -0.185
RMSE: 2.988, R^2: 0.376, MBE: 0.533
RMSE: 2.730, R^2: 0.343, MBE: -0.125
Params: {'num_leaves': 5, 'objective': 'regression', 'min_data_in_leaf': 2, 'learning_rate': 0.27272902895065526, 'n_estimators': 291, 'n_jobs': -1}
Score: 2.9130656595103717
------ Sampling new data point ------
RMSE: 3.254, R^2: 0.275, MBE: -0.018
RMSE: 2.492, R^2: 0.443, MBE: 0.312
RMSE: 3.171, R^2: 0.334, MBE: -0.509
Params: {'num_leaves': 8, 'objective': 'regression', 'min_data_in_leaf': 5, 'learning_rate': 0.836095155661024, 'n_estimators': 235, 'n_jobs': -1}
Score: 2.972315802170764
------ Sa

------ Sampling new data point ------
RMSE: 3.105, R^2: 0.344, MBE: 0.114
RMSE: 2.675, R^2: 0.472, MBE: -0.309
RMSE: 2.405, R^2: 0.528, MBE: 0.333
Params: {'num_leaves': 2, 'objective': 'regression', 'min_data_in_leaf': 2, 'learning_rate': 0.5814058328057227, 'n_estimators': 500, 'n_jobs': -1}
Score: 2.728218485590508
------ Sampling new data point ------
RMSE: 2.580, R^2: 0.492, MBE: -0.228
RMSE: 2.487, R^2: 0.444, MBE: 0.151
RMSE: 2.732, R^2: 0.423, MBE: 0.493
Params: {'num_leaves': 2, 'objective': 'regression', 'min_data_in_leaf': 10, 'learning_rate': 0.33627905720372203, 'n_estimators': 500, 'n_jobs': -1}
Score: 2.5996647976330016
------ Sampling new data point ------
RMSE: 2.956, R^2: 0.403, MBE: -0.285
RMSE: 2.398, R^2: 0.532, MBE: -0.213
RMSE: 2.479, R^2: 0.465, MBE: 0.313
Params: {'num_leaves': 2, 'objective': 'regression', 'min_data_in_leaf': 5, 'learning_rate': 0.4137678817336856, 'n_estimators': 500, 'n_jobs': -1}
Score: 2.6110416322395267
------ Sampling new data point ----

RMSE: 2.752, R^2: 0.444, MBE: 0.135
RMSE: 1.619, R^2: 0.720, MBE: -0.369
RMSE: 4.405, R^2: 0.169, MBE: 0.172
Params: {'max_depth': 16, 'min_samples_leaf': 13, 'min_samples_split': 13, 'n_estimators': 439, 'n_jobs': -1}
Score: 2.9253038789121852
------ Sampling new data point ------
RMSE: 4.029, R^2: 0.152, MBE: 0.201
RMSE: 2.256, R^2: 0.552, MBE: 0.199
RMSE: 2.752, R^2: 0.540, MBE: -0.348
Params: {'max_depth': 16, 'min_samples_leaf': 7, 'min_samples_split': 6, 'n_estimators': 123, 'n_jobs': -1}
Score: 3.01215364214089
------ Sampling new data point ------
RMSE: 2.369, R^2: 0.511, MBE: 0.112
RMSE: 4.288, R^2: 0.199, MBE: 0.239
RMSE: 2.620, R^2: 0.442, MBE: -0.231
Params: {'max_depth': 8, 'min_samples_leaf': 8, 'min_samples_split': 13, 'n_estimators': 292, 'n_jobs': -1}
Score: 3.0923962808979613
------ Sampling new data point ------
RMSE: 2.197, R^2: 0.559, MBE: -0.120
RMSE: 2.725, R^2: 0.475, MBE: -0.125
RMSE: 4.148, R^2: 0.176, MBE: 0.228
Params: {'max_depth': 11, 'min_samples_leaf': 1

RMSE: 2.298, R^2: 0.529, MBE: -0.310
Params: {'max_depth': 25, 'min_samples_leaf': 15, 'min_samples_split': 15, 'n_estimators': 372, 'n_jobs': -1}
Score: 2.9870984822902966
------ Sampling new data point ------
RMSE: 2.307, R^2: 0.543, MBE: -0.068
RMSE: 4.167, R^2: 0.214, MBE: 0.525
RMSE: 2.597, R^2: 0.461, MBE: -0.381
Params: {'max_depth': 2, 'min_samples_leaf': 15, 'min_samples_split': 12, 'n_estimators': 414, 'n_jobs': -1}
Score: 3.0235827354061002
------ Sampling new data point ------
RMSE: 2.724, R^2: 0.461, MBE: -0.486
RMSE: 2.297, R^2: 0.554, MBE: 0.255
RMSE: 4.299, R^2: 0.112, MBE: 0.287
Params: {'max_depth': 2, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 500, 'n_jobs': -1}
Score: 3.106546836228419
------ Sampling new data point ------
RMSE: 2.637, R^2: 0.430, MBE: -0.294
RMSE: 2.217, R^2: 0.593, MBE: -0.066
RMSE: 4.434, R^2: 0.128, MBE: 0.478
Params: {'max_depth': 25, 'min_samples_leaf': 2, 'min_samples_split': 15, 'n_estimators': 312, 'n_jobs': -1}
Score: 

/Users/pluto/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


...

------ Sampling new data point ------
RMSE: 2.290, R^2: 0.474, MBE: -0.144
RMSE: 3.715, R^2: 0.430, MBE: 0.280
RMSE: 2.747, R^2: 0.477, MBE: -0.057
Params: {'hidden_layer_sizes': 9, 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 1.0, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'n_jobs': -1}
Score: 2.917247563766321
------ Sampling new data point ------
RMSE: 2.993, R^2: 0.264, MBE: 0.508
RMSE: 3.226, R^2: 0.399, MBE: -0.281
RMSE: 3.827, R^2: 0.231, MBE: 0.223
Params: {'hidden_layer_sizes': 10, 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 0.6932120746389935, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'n_jobs': -1}
Score: 3.348853112226351
------ Sampling new data point ------
RMSE: 2.222, R^2: 0.533, MBE: -0.748
RMSE: 3.686, R^2: 0.313, MBE: 0.163
RMSE: 3.511, R^2: 0.312, MBE: 0.728
Params: {'hidden_layer_sizes': 9, 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 0.45066157884972396, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'n_jobs': -1}

/Users/pluto/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


------ Sampling new data point ------
RMSE: 3.065, R^2: 0.426, MBE: -0.100
RMSE: 3.197, R^2: 0.362, MBE: 0.089
RMSE: 3.114, R^2: 0.301, MBE: 0.589
Params: {'hidden_layer_sizes': 9, 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 1.0, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'n_jobs': -1}
Score: 3.1251489067822953


/Users/pluto/anaconda3/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


------ Sampling new data point ------
RMSE: 2.827, R^2: 0.463, MBE: 0.043
RMSE: 2.546, R^2: 0.462, MBE: -0.020
RMSE: 4.100, R^2: 0.156, MBE: 0.576
Params: {'hidden_layer_sizes': 9, 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 1.0, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'n_jobs': -1}
Score: 3.157328842145283
------ Sampling new data point ------
RMSE: 4.011, R^2: 0.257, MBE: 0.238
RMSE: 2.668, R^2: 0.394, MBE: 0.081
RMSE: 2.894, R^2: 0.433, MBE: -0.137
Params: {'hidden_layer_sizes': 9, 'activation': 'relu', 'solver': 'lbfgs', 'alpha': 0.2669107878282821, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'n_jobs': -1}
Score: 3.191215202754107
Optimal parameters
Param: hidden_layer_sizes, value: 9
Param: activation, value: relu
Param: solver, value: lbfgs
Param: alpha, value: 1.0
Param: learning_rate_init, value: 0.1
Param: learning_rate, value: adaptive
Layer1 LASSO: Data: subset6

Lasso score: 0.7910 (+/-0.1150)



In [53]:
SCORES

{'Layer1_LGBM': {'rmse': 2.865381963565911,
  'rsqr': 0.3967733829870421,
  'mbe': 0.10037111395540783},
 'Layer1_RFE': {'rmse': 3.0591699520014477,
  'rsqr': 0.38975648459766304,
  'mbe': 0.005150144489836891},
 'Layer1_SVM': {'rmse': 2.3907602431990864,
  'rsqr': 0.5285686414524212,
  'mbe': 0.05222580295385045},
 'Layer1_GP': {'rmse': 3.81923266517254,
  'rsqr': 0.019332333774905137,
  'mbe': -0.20695735254556746},
 'Layer1_ANN': {'rmse': 3.191215202754107,
  'rsqr': 0.3612955630651582,
  'mbe': 0.06070441616445998}}

In [54]:
layer2_filter = (full_df['Set_rank']=='layer2')
X_train_layer2 = full_df.loc[layer2_filter & (~full_df[yvar].isna()), Xvar]


test_filter = (full_df['Set_rank']=='test')
X_test_layer2 = full_df.loc[test_filter & (~full_df[yvar].isna()), Xvar]

In [55]:
# Saving prediction for layer 2 using model trained on layer 1
for key in solvers_layer1:
    val = model_library[key]
    subset = val['data']
    reg = val['model_instance_layer1']

    y_pred_layer2 = reg.predict(X_train_layer2)
    full_df.loc[layer2_filter & (~full_df[yvar].isna()), 
                yvar + f'_predicted_layer2_{key}'] =  y_pred_layer2*ystd + ymean
    
    y_pred_test = reg.predict(X_test_layer2)
    full_df.loc[test_filter & (~full_df[yvar].isna()), 
                yvar + f'_predicted_test_{key}'] =  y_pred_test*ystd + ymean

In [56]:
Xvar_pred_layer2 = [yvar + f'_predicted_layer2_{j}' for j in solvers_layer1]
X_train_pred_layer2 = full_df.loc[layer2_filter & (~full_df[yvar].isna()), Xvar_pred_layer2]
y_train_layer2 = full_df.loc[layer2_filter & (~full_df[yvar].isna()), yvar]

In [59]:
# Ensemble run for solvers in layer 2, ie. using Xtrain = ypredicted from layer 1 models
for key in solvers_layer2:
    label = f'val_Layer2_ensemble_{key}'
    val = deepcopy(model_library[key])
    reg = val['model_instance']
    params_space = val['params_space']

    # Bayesian opt. part
    @use_named_args(params_space)
    def ith_objective(**params):
        cls = reg.set_params(**params)
        return objective_core(cls, X_train_pred_layer2, y_train_layer2, label, **params)

    res = gp_minimize(ith_objective, params_space, n_calls=N_CALLS, random_state=0)
    "Best score=%.4f" % res.fun

    # Generating final optimized model instance
    print("Optimal parameters")
    params = {}
    for param, value in zip(params_space, res.x):
        print(f"Param: {param.name}, value: {value}")
        params[param.name] = value

    ith_model = reg.set_params(**params)
    ith_model.fit(X_train_pred_layer2.values, y_train_layer2.values)
    
    # Model instance for ensemble
    model_library[key]['model_instance_ensemble'] = ith_model

Finished loading model, total used 500 iterations
------ Sampling new data point ------
RMSE: 3.831, R^2: 0.236, MBE: 0.018
RMSE: 3.794, R^2: 0.245, MBE: -0.030
RMSE: 3.741, R^2: 0.229, MBE: 0.179
Params: {'num_leaves': 7, 'objective': 'regression', 'min_data_in_leaf': 9, 'learning_rate': 0.8472670136102473, 'n_estimators': 349, 'n_jobs': -1}
Score: 3.788699792026105
------ Sampling new data point ------
RMSE: 3.408, R^2: 0.275, MBE: -0.066
RMSE: 3.414, R^2: 0.317, MBE: 0.115
RMSE: 3.297, R^2: 0.328, MBE: 0.014
Params: {'num_leaves': 5, 'objective': 'regression', 'min_data_in_leaf': 2, 'learning_rate': 0.27272902895065526, 'n_estimators': 291, 'n_jobs': -1}
Score: 3.3730117897105125
------ Sampling new data point ------
RMSE: 3.805, R^2: 0.255, MBE: -0.008
RMSE: 3.609, R^2: 0.299, MBE: 0.221
RMSE: 3.750, R^2: 0.234, MBE: -0.004
Params: {'num_leaves': 8, 'objective': 'regression', 'min_data_in_leaf': 5, 'learning_rate': 0.836095155661024, 'n_estimators': 235, 'n_jobs': -1}
Score: 3.7216

------ Sampling new data point ------
RMSE: 2.789, R^2: 0.414, MBE: 0.079
RMSE: 2.440, R^2: 0.529, MBE: -0.292
RMSE: 3.633, R^2: 0.336, MBE: 0.217
Params: {'num_leaves': 2, 'objective': 'regression', 'min_data_in_leaf': 10, 'learning_rate': 0.14760767501412292, 'n_estimators': 100, 'n_jobs': -1}
Score: 2.953897758434239
------ Sampling new data point ------
RMSE: 2.928, R^2: 0.413, MBE: 0.018
RMSE: 3.161, R^2: 0.368, MBE: 0.149
RMSE: 2.991, R^2: 0.419, MBE: -0.112
Params: {'num_leaves': 2, 'objective': 'regression', 'min_data_in_leaf': 2, 'learning_rate': 0.1705735669168016, 'n_estimators': 100, 'n_jobs': -1}
Score: 3.0265851562835855
------ Sampling new data point ------
RMSE: 2.780, R^2: 0.401, MBE: 0.166
RMSE: 2.949, R^2: 0.444, MBE: -0.240
RMSE: 3.372, R^2: 0.358, MBE: 0.094
Params: {'num_leaves': 2, 'objective': 'regression', 'min_data_in_leaf': 10, 'learning_rate': 0.17144876827476555, 'n_estimators': 100, 'n_jobs': -1}
Score: 3.0334326152449194
------ Sampling new data point ---

In [60]:
# Final ENSEMBLE prediction on Xtest
X_test_pred_layer2 = full_df.loc[test_filter, Xvar_pred_layer2]

# Prediction
for key in solvers_layer2:
    val = model_library[key]    
    cls = val[f'model_instance_ensemble']
    y_predicted_test = cls.predict(X_test_pred_layer2)
    full_df.loc[test_filter, 
                yvar + f'_predicted_test_ensemble_{key}'] =  y_predicted_test * ystd + ymean

In [61]:
X_train_pred_layer2.shape

(2133, 6)

In [62]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [63]:
# LSTM
NSTEPS = 5
NFEATURES = X_train_pred_layer2.shape[1]

# convert into input/output sequences
dataset_train = np.column_stack((X_train_pred_layer2, y_train_layer2))
dataset_trainX, dataset_trainy = split_sequences(dataset_train, NSTEPS)
print(dataset_trainX.shape, dataset_trainy.shape)

# define model
model = Sequential()
model.add(LSTM(5, input_shape=(NSTEPS, NFEATURES)))
model.add(Dense(4, kernel_initializer='normal', activation='relu'))
model.add(Dense(1))    
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(dataset_trainX, dataset_trainy, 
                    validation_split=0.3, shuffle=False, 
                    epochs=100, batch_size=100, verbose=2)

NameError: name 'split_sequences' is not defined

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
# Single model run
Xtrain_bothlayers = full_df.loc[(full_df['Set_rank']!='test') & (~full_df[yvar].isna()), Xvar]
ytrain_bothlayers = full_df.loc[(full_df['Set_rank']!='test') & (~full_df[yvar].isna()), yvar]

for key in solvers_layer2:
    val = deepcopy(model_library[key])
    reg = val['model_instance']
    params_space = val['params_space']

    # Bayesian opt. part
    @use_named_args(params_space)
    def jth_objective(**params):
        cls = reg.set_params(**params)
        return objective_core(cls, Xtrain_bothlayers, ytrain_bothlayers, **params)

    res = gp_minimize(jth_objective, params_space, n_calls=N_CALLS, random_state=0)
    "Best score=%.4f" % res.fun

    # Generating final optimized model instance
    print("Optimal parameters")
    params = {}
    for param, value in zip(params_space, res.x):
        print(f"Param: {param.name}, value: {value}")
        params[param.name] = value

    jth_model = reg.set_params(**params)
    jth_model.fit(Xtrain_bothlayers.values, ytrain_bothlayers.values)
    
    # Model instance for ensemble
    model_library[key]['model_instance_single'] = jth_model

In [ ]:
# Final SINGLE prediction on Xtest
X_test = full_df.loc[test_filter, Xvar]

# Prediction
for key in solvers_layer2:
    val = model_library[key]    
    cls = val[f'model_instance_single']
    y_predicted_test = cls.predict(X_test)
    full_df.loc[test_filter, yvar + f'_predicted_test_single_{key}'] =  y_predicted_test * ystd + ymean

In [ ]:
full_df.to_csv('../data_out/temp_full.csv')

In [ ]:
full_df.info()

In [ ]:
full_df['Set_rank'].value_counts()

In [ ]:
ytest = full_df.loc[test_filter, yvar] * ystd + ymean

for key in solvers_layer2:
    for j in ['single', 'ensemble']:
        ytest_predicted = full_df.loc[test_filter, yvar + f'_predicted_test_{j}_{key}']

        print('Layer2', key, j)
        ascore = diagnostic_stats(ytest, ytest_predicted)
        #print(ascore)
        SCORES['Layer2' + '_' + key + '_' + j] = ascore

In [ ]:
pd.DataFrame.from_dict(SCORES).T.round(3)

In [ ]:
def diagnostic_stats(ytrue, ypred):
    """
    https://stats.stackexchange.com/questions/142248/difference-between-r-square-and-rmse-in-linear-regression
    
    https://www.sciencedirect.com/topics/engineering/mean-bias-error
    """
    n = len(ytrue)

    # Check that the ytrue and ypred are equal length vector.
    assert n == len(ypred)
    
    # sum squared error
    sse = np.sum((ytrue - ypred)**2)
    
    # root mean square error
    rmse = np.sqrt(sse/n)

    # total sum of squares
    tss = np.sum((ytrue - np.mean(ytrue))**2)
    tst = np.sum((ypred - np.mean(ypred))**2)
    tstp = tst**0.5
    tssp = tss**0.5
    
    soorat = np.sum((ytrue-np.mean(ytrue))*(ypred-np.mean(ypred)))
    
    # Rsquare
    ##rsqr = 1 - sse/tss
    rsqr = (soorat/(tssp*tstp))**2

    # Mean biased error
    mbe = np.mean(ytrue - ypred)
    
    print("RMSE: %1.3f, R^2: %1.3f, MBE: %1.3f"%(rmse, rsqr, mbe))
    
    return {'rmse':rmse, 'rsqr':rsqr, 'mbe':mbe}